#### 泰坦尼克号乘客数据查验

In [5]:
# 导入 pandas 用于数据查验
import pandas as pd
# 利用pandas的read_csv模块直接从互联网收集泰坦尼克号乘客数据。
titanic=pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')
# 观察前几行数据，可以发现，数据类型各异，数值型，类别型，甚至还有缺失数据。
titanic.head()
# 使用pandas,数据都转入pandas独有的dataframe格式（二维数据表格），直接使用info(),查看数据的统计特性。
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


#### 使用决策树模型预测泰坦尼克号乘客的生还情况

In [6]:
# 机器学习有一个不太被初学者重视并且耗时，但是十分重要的一环——特征选择，这个需要基于一些背景知识。
# 根据我们对这场事故的了解，sex,age,pclass这些特征都有可能是决定幸免与否的关键因素。
X=titanic[['pclass','age','sex']]
y=titanic['survived']

# 对当前选择的特征进行探查
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.8+ KB


In [7]:
# 借由上面的输出，我们设计如下几个数据处理的任务；
# 1） age这个数据列，只有633个，需要补完。
# 2） sex与pclass两个数据列的值都都是类别型的，需要转化为数值特征，用0/1代替。
# 首先我们补充age里的数据，使用平均数或者中位数都是对模型偏离造成最小影响的策略。
X['age'].fillna(X['age'].mean(), inplace=True)
# 对补充的数据重新探查
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.8+ KB


/home/qhg/.conda/envs/py2/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
# 由此得知，age特征得到了补充。
# 数据分割
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=33)
# 使用scikit-learn.feature_extraction中的特征转换器，详见3.3.3.3特征抽取。
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer(sparse=False)
# 转换特征后，我们发现凡是类型特征的特征都单独剥离出来，独成一列特征，数值型的则保持不变。
X_train=vec.fit_transform(X_train.to_dict(orient='record'))
print vec.feature_names_

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


In [12]:
# 同样需要对测试数据的特征进行转换。
X_test=vec.transform(X_test.to_dict(orient='record'))
# 从sklearn.tree中导入决策树分类器。
from sklearn.tree import DecisionTreeClassifier
# 使用默认配置初始化决策树分类器。
dtc=DecisionTreeClassifier()
# 使用分割到的训练数据进行模型学习。
dtc.fit(X_train, y_train)
# 用训练好的决策树模型对测试特征数据进行预测。
y_predict=dtc.predict(X_test)

#### 决策树模型对泰坦尼克号乘客是否生还的预测性能

In [14]:
# 从sklearn.metrics导入classification_report。
from sklearn.metrics import classification_report
# 输出预测准确性。
print dtc.score(X_test, y_test)
# 输出更加详细的分类性能。
print classification_report(y_predict, y_test, target_names=['died', 'survived'])

0.7811550151975684
             precision    recall  f1-score   support

       died       0.91      0.78      0.84       236
   survived       0.58      0.80      0.67        93

avg / total       0.81      0.78      0.79       329

